In [ ]:
import numpy as np
import glob
import sys
from tqdm.auto import tqdm
import os
import random

In [ ]:
# Selected synthetic occlusions
# so if glass and mask => glass = 1 and mask = 1 and cap = 0
mask = 0
glass = 0
sunglass = 0
cap = 1

# Paths used in the script
embeddings_synthetic_images = './embeddings/synthetic/'
scores_path_root = "./similarity_scores/"

In [ ]:
# Creating filter based on selected occlusions
select_occlusions = ''
non_selected_occlusions = ['_mask', '_glass', '_sunglass', '_cap']
if mask:
    select_occlusions = '_mask'
    non_selected_occlusions.remove('_mask')
if glass:
    select_occlusions = select_occlusions + '_glass'
    non_selected_occlusions.remove('_glass')
if sunglass:
    select_occlusions = select_occlusions + '_sunglass'
    non_selected_occlusions.remove('_sunglass')
if cap:
    select_occlusions = select_occlusions + '_cap'
    non_selected_occlusions.remove('_cap')

In [ ]:
file_names = [os.path.basename(file) for file in glob.glob(embeddings_synthetic_images + '*.npy')]
filtered_file_names = []
for file in file_names:
    if select_occlusions in file and not any(str in file for str in non_selected_occlusions):
        filtered_file_names.append(file)

In [ ]:
def similarity(embeddings1, embeddings2):
    embeddings1=embeddings1.astype(np.float64)
    embeddings2=embeddings2.astype(np.float64)
    # Cosine similarity
    dot = np.sum(np.multiply(embeddings1, embeddings2), axis=0)
    norm = np.linalg.norm(embeddings1, axis=0) * np.linalg.norm(embeddings2, axis=0)
    similarity = dot/norm
    similarity = min(1,similarity)
    return similarity

In [ ]:
# Synthetic images

mated_comp_count = 0
non_mated_comp_count = 0

synthetic_images_mated = np.zeros(0, dtype=float, order='C')
synthetic_images_non_mated = np.zeros(0, dtype=float, order='C')

ids = [file.split('d')[0] for file in filtered_file_names]
ids = [*set(ids)]

for id in tqdm(ids, file=sys.stdout):
    images_of_id = glob.glob(embeddings_synthetic_images + id + '*')
    filtered_images_of_id = []
    for image in images_of_id:
        if select_occlusions in image and not any(str in image for str in non_selected_occlusions):
          filtered_images_of_id.append(image)
    reference_feature = random.choice(filtered_images_of_id)
    reference_feature_embedding = np.load(reference_feature)
    for file in filtered_file_names:
        file_path = embeddings_synthetic_images + file
        if file.split('d')[0] == id: # Mated comparison
            if not (file_path == reference_feature):
                mated_comp_count += 1
                mated_feature_embedding = np.load(file_path)
                synthetic_images_mated = np.append(synthetic_images_mated,
                                              similarity(reference_feature_embedding, mated_feature_embedding))
        else: # Non-mated comparison
            non_mated_comp_count += 1
            non_mated_feature_embedding = np.load(file_path)
            synthetic_images_non_mated = np.append(synthetic_images_non_mated,
                                              similarity(reference_feature_embedding, non_mated_feature_embedding))            

print("Synthetic mated comparisons:", mated_comp_count)
print("Synthetic non-mated comparisons:", non_mated_comp_count)

In [ ]:
# Saving the similarity score if scores exist
if select_occlusions:
    if (len(synthetic_images_mated) == 0 and len(synthetic_images_non_mated) == 0):
        print(
            "No embeddings for the given occlusions found in the directories",
            file=sys.stderr)
    else:
        if not os.path.isdir(scores_path_root):
            os.makedirs(scores_path_root)

        if len(synthetic_images_mated) == 0:
            print("No mated comparisons for the given occlusions",
                  file=sys.stderr)
        else:
            np.save((scores_path_root + "synthetic_images_mated"
                     + select_occlusions + ".npy"), synthetic_images_mated)

        if len(synthetic_images_non_mated) == 0:
            print("No non-mated comparisons for the given occlusions",
                  file=sys.stderr)
        else:
            np.save((scores_path_root + "synthetic_images_non-mated"
                     + select_occlusions + ".npy"), synthetic_images_non_mated)

else:
    print("Run terminated because occlusions is not selected", file=sys.stderr)